## Этапы (простой) обработки текста

<img src="images/textm.png">


## Декодирование


**Def.**  
перевод последовательности байт в последовательность символов

* Распаковка  
*plain/.zip/.gz/...*
* Кодировка  
*ASCII/utf-8/Windows-1251/...*
* Формат  
*csv/xml/json/doc...*

Кроме того: что такое документ?



## Разбиение на токены
**Def.**  
разбиение последовательности символов на части (токены), возможно, исключая из рассмотрения некоторые символы  
Наивный подход: разделить строку пробелами и выкинуть знаки препинания  


*Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.*  


**Проблемы:**  
* example@example.com, 127.0.0.1
* С++, C#
* York University vs New York University
* Зависимость от языка (“Lebensversicherungsgesellschaftsangestellter”, “l’amour”)
Альтернатива: n-граммы

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from nltk.tokenize import RegexpTokenizer


s = "Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру."

tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
for t in tokenizer.tokenize(s): 
    print(t)

Трисия
любила
Нью
-
Йорк
,
поскольку
любовь
к
Нью
-
Йорку
могла
положительно
повлиять
на
ее
карьеру
.


## Стоп-слова
**Def.**  
Наиболее частые слова в языке, не содержащие никакой информации о содержании текста



In [ ]:
from nltk.corpus import stopwords


print(" ".join(stopwords.words("russian")[1:20]))

Проблема: “To be or not to be"

## Нормализация
**Def.**  
Приведение токенов к единому виду для того, чтобы избавиться от поверхностной разницы в написании  

Подходы  
* сформулировать набор правил, по которым преобразуется токен  
Нью-Йорк → нью-йорк → ньюйорк → ньюиорк
* явно хранить связи между токенами (WordNet – Princeton)  
машина → автомобиль, Windows 6→ window

In [ ]:
s = "Нью-Йорк"
s1 = s.lower()
print(s1)

In [ ]:
import re
s2 = re.sub(r"\W", "", s1, flags=re.U)
print(s2)

In [ ]:
s3 = re.sub(r"й", u"и", s2, flags=re.U)
print(s3)

## Стемминг и Лемматизация
**Def.**  
Приведение грамматических форм слова и однокоренных слов к единой основе (lemma):
* Stemming – с помощью простых эвристических правил
  * Porter (Cambridge – 1980)
        5 этапов, на каждом применяется набор правил, таких как
            sses → ss (caresses → caress)
            ies → i (ponies → poni)

  * Lovins (1968)
  * Paice (1990)
  * другие
* Lemmatization – с использованием словарей и морфологического анализа


## Стемминг

In [ ]:
from nltk.stem.snowball import PorterStemmer
from nltk.stem.snowball import RussianStemmer


s = PorterStemmer()
print(s.stem("Tokenization"))
print(s.stem("stemming"))

r = RussianStemmer()
print(r.stem("Авиация"))
print(r.stem("национальный"))

**Наблюдение**  
для сложных языков лучше подходит лемматизация

## Лемматизация

In [5]:
%pip install pymorphy2

     -------------------------------------- 55.5/55.5 kB 580.0 kB/s eta 0:00:00
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.2/8.2 MB 5.3 MB/s eta 0:00:00
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
  Running setup.py install for docopt: started
  Running setup.py install for docopt: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()
print(morph.parse("думающему")[0].normal_form)

## Heaps' law
Эмпирическая закономерность в лингвистике, описывающая распределение числа уникальных слов в документе (или наборе документов) как функцию от его длины.

$$
M = k T^\beta, \;M \text{ -- размер словаря}, \; T \text{ -- количество слов в корпусе}
$$
$$
30 \leq k \leq 100, \; b \approx 0.5
$$

<img src="images/dim.png">
<img src="images/heaps.png">

## Представление документов
**Boolean Model.** Присутствие или отсутствие слова в документе  
**Bag of Words.** Порядок токенов не важен  

*Погода была ужасная, принцесса была прекрасная.
Или все было наоборот?*

Координаты
* Мультиномиальные: количество токенов в документе
* Числовые: взвешенное количество токенов в документе

## Zipf's law
Эмпирическая закономерность распределения частоты слов естественного языка

$t_1, \ldots, t_N$ - токены, отранжированные по убыванию частоты
   	
$f_1, \dots, f_N$ - соответствующие частоты

**Закон Ципфа**
	$$
	f_i = \frac{c}{i^k}
	$$	
	
	Что еще? Посещаемость сайтов, количество друзей, население городов...
<img src="images/zipf.png">


In [ ]:
import pandas as pd
s=pd.Series(["Мама мыла раму мылом", "У попа была собака он её любил"],dtype=str)
# s = s.apply(lambda x: x.lower())
s=s.str.lower()
s=s.str.strip()
s
nn=set()
def growset(x):
    for w in x.split():
        nn.add(w)
s.apply(growset)        
df=pd.DataFrame(data=[(list(nn))],columns=[f'col{i}' for i in range(len(nn))])
df

list(nn)
[(list(nn))]
s = s.str.split(" ", expand=True)
s


In [4]:
%pip install morpher 


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement morpher (from versions: none)
ERROR: No matching distribution found for morpher


In [7]:
import string
import pymorphy2
morpher = pymorphy2.MorphAnalyzer()

def preprocess_txt(line):
    sw=[]
    # Почистим строку от пунктуации. Для этого пробежимся по каждому символу и проверим, не является ли он знаком пунктуации
    exclude = set(string.punctuation)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # Лемматизируем все слова в нашем тексте
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
import pandas as pd
s = pd.Series(["Мама мыла раму мылом", "У попа была собака он её любил"], dtype="string")
s = s.apply(lambda x: preprocess_txt(x))
s

0                  [мама, мыло, рама, мыло]
1    [у, поп, быть, собака, он, её, любить]
dtype: object

In [1]:
%pip install gensim

     ---------------------------------------- 23.9/23.9 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 38.6/38.6 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 4.9 MB/s eta 0:00:00
     -------------------------------------- 983.8/983.8 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 58.6/58.6 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведем слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

[==================================================] 100.0% 128.1/128.1MB downloaded
cereal


In [3]:
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

cereal
